### Reproduction de l'exemple de l'article `Taming Model Serving Complexity, Performance and Cost: A Compilation to Tensor Computations Approach`.

In [1]:
import numpy as np

In [2]:
def compute_path(children_left, children_right, k):
    """Compute path from nodes (leaf or internal) to root.
    Args:
        tree
        k (int) : index from start
    Return:
        List of [id, 1] if LeftSubTree, [id, -1] if RightSubTree
    """
    l = [[k,0]]
    left = children_left
    right = children_right

    while l[-1][0] != 0:
        parent_left = np.flatnonzero((left == l[-1][0]),)
        parent_right = np.flatnonzero((right == l[-1][0]),)

        if len(parent_left) > 0:
            l.append([parent_left[0], 1])
        else:
            l.append([parent_right[0], -1])

    return l[1:]

In [3]:
X = [0.1, 4.6, 1.9, 0.8, 3.5]

In [4]:
children_left  = np.array([1, 2, -1, -1, 5, 6, -1, -1, -1])
children_right = np.array([4, 3, -1, -1, 8, 7, -1, -1, -1])
feature = np.array([2, 1, -2, -2, 4, 2, -2, -2, -2])
threshold = np.array([0.5, 2.0, -2, -2, 5.5, 2.4, -2, -2, -2])
n_features = 5

In [5]:
is_internal_nodes = children_left != children_right
internal_nodes = np.flatnonzero(is_internal_nodes)
leaf_nodes = np.flatnonzero(~is_internal_nodes)
split_features_internal_nodes = feature[internal_nodes]

In [6]:
split_features_internal_nodes

array([2, 1, 4, 2])

In [7]:
# Matrix A
A = np.zeros(shape=(n_features, len(internal_nodes)))
for i,j in enumerate(split_features_internal_nodes):
    A[j,i] = 1
A

array([[0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.]])

In [8]:
# Matrix B
B = threshold[internal_nodes]
B

array([0.5, 2. , 5.5, 2.4])

In [9]:
# Matrix C
sub_to_global_internal_nodes = { v:k for k,v in enumerate(internal_nodes) }
s_to_g = lambda x : sub_to_global_internal_nodes[x]
C = np.zeros(shape=(len(internal_nodes), len(leaf_nodes)))

for j, leaf_idx in enumerate(leaf_nodes):
    path = compute_path(children_left, children_right, leaf_idx)

    for i in range(len(path)):
        # Apply transformation on index
        # subset (internal nodes)
        #      -> global (internal and leaf nodes)
        path[i][0] = s_to_g(path[i][0])

    for node_idx, value in path:
        C[node_idx,j] = value
C

array([[ 1.,  1., -1., -1., -1.],
       [ 1., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  1., -1.],
       [ 0.,  0.,  1., -1.,  0.]])

In [10]:
# Matrix D
D = np.sum(C == 1, axis=0)
D

array([2, 1, 2, 1, 0])

In [11]:
# Matrix E
E = np.zeros(shape=(len(leaf_nodes), 2))
leaf_to_class = [0, 1, 1, 0, 0]

for i in range(len(leaf_nodes)):
    E[i,leaf_to_class[i]] = 1
E

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [12]:
T = (X@A) < B
T

array([False, False,  True,  True])

In [13]:
T = (T@C) == D
T

array([False, False,  True, False, False])

In [14]:
T@E

array([0., 1.])